아이템 기반 필터링 구현하기

- 비회원 장르별 평점 순위 보여주기 기능 구현(장르별 네이버 평점 보여주기)
- 사용자 선호에 따른 장르, 감독별 유사도 검증 구현 

In [147]:
##########################################

# 비회원 장르별 순위 추천


In [3]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select m.*, g.*
    from movie m
    left outer join moviegenre mg
    on m.moviecd = mg.moviecd
    left outer join genre g
    on mg.genrecd = g.genrecd
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_genres = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd', 'genreCd', 'genreNm'])
movie_genres

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreCd,genreNm
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁
1,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,13,서부극(웨스턴)
2,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,19,드라마
3,19188001,개의 일생,A Dog's Life,None,1918,None,0,None,None,None,7,코미디
4,19190002,여름밤의 기묘한 이야기,Unheimliche Geschichten,None,1919,None,0,None,None,None,6,공포(호러)
...,...,...,...,...,...,...,...,...,...,...,...,...
73025,20219978,볼레로 만들기,Structure: Making Bolero,None,2020,None,0,None,None,None,4,기타
73026,20219984,어디에도 없는 시간,Where would we go,None,2021,None,0,None,None,None,19,드라마
73027,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,7,코미디
73028,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마


In [4]:
# 2. movie_genres 데이터 전처리
# 2-1. naverScore 데이터 결측치 0으로 대체
movie_genres["naverScore"] = movie_genres['naverScore'].fillna(0)

# 2-2. not-null 평균값으로 대체하기
movie_genres2 = movie_genres.groupby(by=["genreNm", "movieNm"])["naverScore"].mean()

# 2-3. 장르별 높은 평점 영화 내림차순 정리
movie_genres2 = movie_genres2.sort_values(ascending=False)
movie_genres2

genreNm  movieNm               
애니메이션    디지몬 어드벤처 라스트 에볼루션 : 인연    4.8
         극장판 바이올렛 에버가든             4.8
드라마      베일리 어게인                   4.8
어드벤처     디지몬 어드벤처 라스트 에볼루션 : 인연    4.8
판타지      극장판 바이올렛 에버가든             4.8
                                  ... 
드라마      어때?                       0.0
         어떤 개인 날                   0.0
         어떤 나들이                    0.0
         어떤 날                      0.0
판타지      히치하이킹                     0.0
Name: naverScore, Length: 70696, dtype: float64

In [5]:
# 3. 판타지 장르 내림차순 뽑기
movie_genres2[['판타지']].sort_values(ascending=False).head(10)

# 비회원 장르별 평점 순위 보여주기 기능 구현

genreNm  movieNm      
판타지      극장판 바이올렛 에버가든    4.8
         센과 치히로의 행방불명     4.7
         트루먼 쇼            4.7
         알라딘              4.7
         지금, 만나러 갑니다      4.7
         메리 포핀스           4.7
         토이 스토리 3         4.7
         천녀유혼             4.6
         미이라              4.6
         미녀와 야수           4.6
Name: naverScore, dtype: float64